In [24]:
# import rclpy
# from rclpy.node import Node
# from ament_index_python.packages import get_package_share_directory
# from sensor_msgs.msg import Image
import numpy as np
import cv2 as cv
from cv_bridge import CvBridge
import matplotlib.pyplot as plt
import os

print(cv2.__version__)


4.11.0


In [25]:
def enhance_contrast_brightness(img):
        # Adjust the brightness and contrast 
        # Adjusts the brightness by adding 10 to each pixel value 
        brightness = 0 
        # Adjusts the contrast by scaling the pixel values by 2.3 
        contrast = 2.3  
        return cv.addWeighted(img, contrast, np.zeros(img.shape, img.dtype), 0, brightness) 
    
def sharpen(img):
    # Create the sharpening kernel 
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]) 
    
    # Sharpen the image 
    return cv.filter2D(img, -1, kernel)

def clarity_enhancement(img):
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))
    topHat = cv.morphologyEx(img, cv.MORPH_TOPHAT, kernel)
    blackHat = cv.morphologyEx(img, cv.MORPH_BLACKHAT, kernel)
    res = img + topHat - blackHat 
    return res

def laplace_sharpen(img):
    return cv.Laplacian(img, cv.CV_64F) 

def histogram_equalization(img):
    eq_img = cv.equalizeHist(img)
    
    hist=cv.calcHist(img,[0],None,[256],[0,256])
    hist1=cv.calcHist(eq_img,[0],None,[256],[0,256])
    plt.subplot(121)
    plt.title("Image1")
    plt.xlabel('bins')
    plt.ylabel("No of pixels")
    plt.plot(hist)
    plt.subplot(122)
    plt.title("Image2")
    plt.xlabel('bins')
    plt.ylabel("No of pixels")
    plt.plot(hist1)
    plt.show()
    
    return eq_img

def callback(msg):
    
    img = bridge.imgmsg_to_cv2(msg)
    # Enhance
    # img = enhance_contrast_brightness(img)
    # img = sharpen(img)
    # img = laplace_sharpen(img)
    img = clarity_enhancement(img)
    
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    # gray = histogram_equalization(gray)

    sift = cv.SIFT_create()
    kp = sift.detect(gray,None)

    img=cv.drawKeypoints(gray,kp,img, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    msg_out = bridge.cv2_to_imgmsg(cvim=img)
    get_logger().info('Processed Image')
    publisher.publish(msg_out)


def test_read():
    image_path = "./image.png"
    
    img = cv.imread(image_path)
    cv.imshow("Window Name", img)
    get_logger().info('Test Image Read!')
    return bridge.cv2_to_imgmsg(cvim=img)

    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)

    sift = cv.SIFT_create()
    kp = sift.detect(gray,None)

    img=cv.drawKeypoints(gray,kp,img)
    return img

In [ ]:
def main(args=None):
    thermal_filter = ThermalFilter()
if __name__ == '__main__':
    main()

In [ ]:
image_path = "./image.png"
img = cv.imread(image_path)
cv.imshow("Window Name", img)

error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
